In [1]:
import warnings
warnings.filterwarnings("ignore")
import os

os.sys.path.insert(0, '../script')

from config import *
from webnlg import *

In [2]:
my_models_paths = !ls -d -1 ../data/models/*.txt

my_models_paths

['../data/models/5 - Model - Template Based - roots_3ae48a5d03a715d609dc8d73f82ac5682cba128d.txt',
 '../data/models/5 - Model - Template Based - roots_4e9245cbbc05e4fdf0de404fbada20d8294127b2.txt',
 '../data/models/5 - Model - Template Based - roots_55f9d82b420c5cbc6a01d5a65082414b87a81195.txt',
 '../data/models/5 - Model - Template Based - roots_ef0aa8122b6669218ec93658c2b7f74042635996.txt']

In [3]:
models_paths = my_models_paths + SUBMISSIONS_FILEPATHS
models_paths

['../data/models/5 - Model - Template Based - roots_3ae48a5d03a715d609dc8d73f82ac5682cba128d.txt',
 '../data/models/5 - Model - Template Based - roots_4e9245cbbc05e4fdf0de404fbada20d8294127b2.txt',
 '../data/models/5 - Model - Template Based - roots_55f9d82b420c5cbc6a01d5a65082414b87a81195.txt',
 '../data/models/5 - Model - Template Based - roots_ef0aa8122b6669218ec93658c2b7f74042635996.txt',
 '../data/webnlg2017/submissions/adaptCenter/ADAPTcentreWebNLGsubmission.txt',
 '../data/webnlg2017/submissions/melbourne/final_result.txt',
 '../data/webnlg2017/submissions/pkuwriter/PKUWriter_results.txt',
 '../data/webnlg2017/submissions/tilburg/nmt_test.out.ordered',
 '../data/webnlg2017/submissions/tilburg/smt_test.out.ordered',
 '../data/webnlg2017/submissions/tilburg/template_test.out.ordered',
 '../data/webnlg2017/submissions/uit-danglt-clnlp/Submission-UIT-DANGNT-CLNLP.txt',
 '../data/webnlg2017/submissions/upf/UPF_All_sent_final.txt',
 '../data/webnlg2017/submissions/baseline_sorted.txt'

In [4]:
model_results = {}

for model_path in models_paths:
    
    with open(model_path) as f:
        
        model_results[model_path] = [line[:-1] for line in f]

In [5]:
import pandas as pd
pd.set_option('max_colwidth', 1000)
from nltk.translate.bleu_score import sentence_bleu

lex = WebNLGCorpus.load("test_with_lex")

s_multiwordexpressions = lex.mdf.m_subject.str.lower().str.replace('_', ' ').str.split().tolist()
o_multiwordexpressions = lex.mdf.m_object.str.lower().str.replace('_', ' ').str.split().tolist()

all_multiwordexpressions = s_multiwordexpressions + o_multiwordexpressions
all_multiwordexpressions = list(set([tuple(s) for s in all_multiwordexpressions]))

from nltk.tokenize.mwe import MWETokenizer

t = MWETokenizer(all_multiwordexpressions)

from textacy.preprocess import preprocess_text

def preprocess(ref):
    
    ref = preprocess_text(ref, no_accents=True, lowercase=True, no_punct=True)
    
    return t.tokenize(ref.split())


def print_entry_and_models(e):
    
    print(e)
    print('\n')
    
    idx = int(e.idx().split('_')[1])
    
    references = [preprocess(ref) for ref in e.ldf.ltext.tolist()]
    
    texts = []
    bleus = []
    
    for model_path in models_paths:
        
        model_text = model_results[model_path][idx]
        
        bleu = sentence_bleu(references, preprocess(model_text))
        
        texts.append(model_text)
        bleus.append(bleu)
        
    models_names = [model_path.split('/')[-1] for model_path in models_paths]
        
    return pd.DataFrame({'models': models_names,
                         'texts': texts,
                         'bleu': bleus}).sort_values('bleu', ascending=False)

In [6]:
test_with_lex = WebNLGCorpus.load('test_with_lex')

In [7]:
entries = list(test_with_lex)

In [8]:
i = 0

In [40]:
e = entries[i]
i += 1

df = print_entry_and_models(e)

my_model_bleu = df[df.models == '5 - Model - Template Based - roots_ef0aa8122b6669218ec93658c2b7f74042635996.txt'].bleu.values[0]

if df.bleu.max() - my_model_bleu > .5:
    
    print('foi')

Triple info: {'category': 'Airport', 'eid': 'Id27', 'idx': '0_26', 'ntriples': 1}

	Modified triples:

Denmark | demonym | Danes


	Lexicalizations:

Denmark demonym is Danes.
The inhabitants of Denmark have the demonym of Danes.
The people of Denmark are called Danes.


foi


In [41]:
df

,bleu,models,texts
4,1.000000e+00,ADAPTcentreWebNLGsubmission.txt,the people of denmark are called danes .
9,1.000000e+00,template_test.out.ordered,denmark demonym is danes .
12,8.091067e-01,baseline_sorted.txt,the people of denmark are called texans .
8,7.071068e-01,smt_test.out.ordered,the people of denmark danes
6,6.147882e-01,PKUWriter_results.txt,the people of denmark are from denmark .
7,4.549941e-01,nmt_test.out.ordered,the inhabitants of denmark are known as danes .
10,9.367929e-155,Submission-UIT-DANGNT-CLNLP.txt,the demonym for people living in the denmark is danes .
3,1.695406e-231,5 - Model - Template Based - roots_ef0aa8122b6669218ec93658c2b7f74042635996.txt,Danes are from Denmark.
0,1.603416e-231,5 - Model - Template Based - roots_3ae48a5d03a715d609dc8d73f82ac5682cba128d.txt,Denmark is inhabited by Danes.
11,1.531972e-231,UPF_All_sent_final.txt,danes live in denmark .


In [21]:
e.get_data()

[{'m_subject': 'Amsterdam_Airport_Schiphol',
  'm_predicate': '5th_runway_SurfaceType',
  'm_object': '"Asphalt"'}]

In [23]:
test_with_lex.sample(idx='0_10')

Triple info: {'category': 'Airport', 'eid': 'Id11', 'idx': '0_10', 'ntriples': 1}

	Modified triples:

Amsterdam_Airport_Schiphol | 5th_runway_SurfaceType | "Asphalt"


	Lexicalizations:

The 5th runway at Amsterdam airport Schiphol has an asphalt surfacing.
The fifth runway of Amsterdam Airport Schiphol has an asphalt surface.
The 5th runway of the Amsterdam Airport Schiphol is asphalt.

<img src='../img/bad_template.jpg' />

Last model verbalization:

The fourth runway at Schiphol Airport in Amsterdam Airport Schuphol Asphalt.